In [1]:
import folium
import glob
import os

from IPython.display import display
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

In [2]:
DATA_DIR = 'example'
IMG_EXT = '.jpg'

GPS_EXIF_TAG = 'GPSInfo'
GPS_LAT_TAG = 'GPSLatitude'
GPS_LONG_TAG = 'GPSLongitude'

JERUSALEM_COORD = [31.7683, 35.2137]
DEFAULT_ZOOM = 7

In [28]:
def load_images(dirname):
    glob_expr = os.path.join(dirname, '*{}'.format(IMG_EXT))
    img_files = glob.glob(glob_expr)
    images = []
    for img_file in img_files:
        img = Image.open(img_file)
        images.append(img)
    return images

def get_gps_img_data(exif_data):
    for tag_id in exif_data:
        # get the tag name, instead of human unreadable tag id
        tag = TAGS.get(tag_id)
        if tag == GPS_EXIF_TAG:
            data = exif_data.get(tag_id)
            return data
    return None

def dms_to_dec(dms):
    # currently handling only N and E
    degree, minute, second = dms
    return int(degree) + float(minute) / 60 + float(second) / 3600

def gps_info_to_coord(gps_info):
    for tag_id, value in gps_info.items():
        tag = GPSTAGS.get(tag_id)
        if tag == GPS_LAT_TAG:
            lat = dms_to_dec(value)
        elif tag == GPS_LONG_TAG:
            long = dms_to_dec(value)
    return lat, long

def get_images_coords(images):
    coords = []
    for img in images:
        exif_data = img.getexif()
        gps_data = get_gps_img_data(exif_data)
        if gps_data is not None and len(gps_data) > 0:
            lat, long = gps_info_to_coord(gps_data)
            coords.append((lat, long))
    return coords

In [29]:
images = load_images(DATA_DIR)

In [30]:
coords = get_images_coords(images)

In [31]:
fmap = folium.Map(location=JERUSALEM_COORD, zoom_start=DEFAULT_ZOOM)
for coord in coords:
    folium.Marker(coord).add_to(fmap)
fmap